In [2]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-125439")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-125439
Azure region: southcentralus
Subscription id: 82648f26-b738-43a4-9ebb-f954c9f1ff3a
Resource group: aml-quickstarts-125439


In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
# https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.compute.amlcompute(class)?view=azure-ml-py#provisioning-configuration-vm-size-----vm-priority--dedicated---min-nodes-0--max-nodes-none--idle-seconds-before-scaledown-none--admin-username-none--admin-user-password-none--admin-user-ssh-key-none--vnet-resourcegroup-name-none--vnet-name-none--subnet-name-none--tags-none--description-none--remote-login-port-public-access--notspecified--

# https://docs.microsoft.com/en-us/azure/machine-learning/how-to-tune-hyperparameters
# https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/ml-frameworks/scikit-learn/train-hyperparameter-tune-deploy-with-sklearn/train-hyperparameter-tune-deploy-with-sklearn.ipynb
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "cpu-cluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                              max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [4]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
ps =  RandomParameterSampling({
    '--C': uniform(0.5, 2),
    '--max_iter': choice(100, 500)
})

# Specify a Policy
policy = BanditPolicy(evaluation_interval=4, slack_factor=0.08)

if "training" not in os.listdir():
    os.mkdir("./training")

# copy the training script train.py into your project directory
import shutil
shutil.copy('train.py', "./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory='.',
compute_target=ws.compute_targets['cpu-cluster'],
entry_script='train.py'
)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,
hyperparameter_sampling=ps,
policy=policy,
primary_metric_name='Accuracy',
primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
max_total_runs=20,
max_concurrent_runs=4)
print('HyperDriveConfig has been created!')

HyperDriveConfig has been created!


In [5]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
from azureml.widgets import RunDetails

my_run = exp.submit(config=hyperdrive_config)
RunDetails(my_run).show()
my_run.wait_for_completion(show_output=True)
assert(my_run.get_status()=="Completed")

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_660ed9e1-2b76-4e7f-8241-deb9c660f7f2
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_660ed9e1-2b76-4e7f-8241-deb9c660f7f2?wsid=/subscriptions/82648f26-b738-43a4-9ebb-f954c9f1ff3a/resourcegroups/aml-quickstarts-125439/workspaces/quick-starts-ws-125439

Streaming azureml-logs/hyperdrive.txt

"<START>[2020-11-09T20:16:09.700809][API][INFO]Experiment created<END>\n""<START>[2020-11-09T20:16:10.405708][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2020-11-09T20:16:10.686976][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2020-11-09T20:16:11.0806451Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>

Execution Summary
RunId: HD_660ed9e1-2b76-4e7f-8241-deb9c660f7f2
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_660ed9e1-2b76-4e7f-8241-deb9c660f7f2?wsid=/subscriptions/82648

In [6]:
import joblib
from azureml.core import Model

# Get your best run and save the model from that run.

best_run = my_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()
run_definition = parameter_values.get("runDefinition")

print('Best Run Id: ', best_run.id)
print('Accuracy:', best_run_metrics['Accuracy'])
print('Arguments for best run:', run_definition.get("arguments"))
print()
print(best_run.get_file_names())
print()
# Register model
model = best_run.register_model(model_name="my_best_run", model_path="outputs/model.joblib")
# Check model
for model in Model.list(ws):
    print("Model Name: {}\n".format(model.name))
    print(model)


Best Run Id:  HD_660ed9e1-2b76-4e7f-8241-deb9c660f7f2_2
Accuracy: 0.9140568099053168
Arguments for best run: ['--C', '1.8920399539831427', '--max_iter', '500']

['azureml-logs/55_azureml-execution-tvmps_0831a102921d0aa14c2ff41ae89002976889e81a0a8c4894febfdf9f89c9f57c_d.txt', 'azureml-logs/65_job_prep-tvmps_0831a102921d0aa14c2ff41ae89002976889e81a0a8c4894febfdf9f89c9f57c_d.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_0831a102921d0aa14c2ff41ae89002976889e81a0a8c4894febfdf9f89c9f57c_d.txt', 'azureml-logs/process_info.json', 'azureml-logs/process_status.json', 'logs/azureml/108_azureml.log', 'logs/azureml/job_prep_azureml.log', 'logs/azureml/job_release_azureml.log', 'outputs/model.joblib']

Model Name: my_best_run

Model(workspace=Workspace.create(name='quick-starts-ws-125439', subscription_id='82648f26-b738-43a4-9ebb-f954c9f1ff3a', resource_group='aml-quickstarts-125439'), name=my_best_run, id=my_best_run:1, version=1, tags={}, properties={})


In [7]:
from azureml.data.dataset_factory import TabularDatasetFactory

# https://docs.microsoft.com/en-us/azure/machine-learning/tutorial-auto-train-models?view=azure-ml-py

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

web_path = ['https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv']
ds = TabularDatasetFactory.from_delimited_files(path=web_path)

In [8]:
from train import clean_data
from sklearn.model_selection import train_test_split
import pandas as pd

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

x_train, x_test, y_train, y_test = train_test_split(x,y)

train_data = pd.concat([x_train, y_train], axis=1)
valid_data = pd.concat([x_test, y_test], axis=1)

In [13]:
# automl_config requires TabularDataset as a result we need to
# create a dataset from pandas dataframe
# https://docs.microsoft.com/en-us/azure/machine-learning/how-to-create-register-datasets#create-a-filedataset
print(type(train_data))

from azureml.core import Workspace, Dataset

# create data folder if not exist 
if "data" not in os.listdir():
    os.mkdir("./data")

# convert train dataframe
train_path = 'data/train_cleaned.csv'
train_data.to_csv(train_path)

# convert valid dataframe
valid_path = 'data/valid_cleaned.csv'
valid_data.to_csv(valid_path)

# get the datastore to upload prepared data
datastore = ws.get_default_datastore()

# upload the local file from src_dir to the target_path in datastore
datastore.upload(src_dir='data', target_path='data')

# create a dataset referencing the cloud location
train_dataset = Dataset.Tabular.from_delimited_files(path = [(datastore, ('data/train_cleaned.csv'))])

# create a dataset referencing the cloud location
valid_dataset = Dataset.Tabular.from_delimited_files(path = [(datastore, ('data/valid_cleaned.csv'))])

    

<class 'pandas.core.frame.DataFrame'>
Uploading an estimated of 2 files
Uploading data/train_cleaned.csv
Uploaded data/train_cleaned.csv, 1 files out of an estimated total of 2
Uploading data/valid_cleaned.csv
Uploaded data/valid_cleaned.csv, 2 files out of an estimated total of 2
Uploaded 2 files


In [16]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    compute_target=ws.compute_targets['cpu-cluster'],
    task="classification",
    primary_metric="accuracy",
    training_data=train_dataset,
    label_column_name='y',
    max_cores_per_iteration=-1, 
    n_cross_validations=5)

In [17]:
# Submit your automl run

exp = Experiment(workspace=ws, name="udacity-project-AutoML")

my_automl_run = exp.submit(config=automl_config)
RunDetails(my_automl_run).show()
my_automl_run.wait_for_completion(show_output=True)
assert(my_automl_run.get_status()=="Completed")

Running on remote.


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…


Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+--------------------------------------+
|Size of the smallest class       |Name/Label of the smallest class |Number of samples in the training data|
+=================================+=====

AssertionError: 

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###